# Setup

In [31]:
import json
import requests
from datetime import datetime as dt
from collections import defaultdict
import pandas as pd

In [32]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display, Markdown

In [33]:
def printjson(j):
    print(json.dumps(j,indent=4))
def print_json(j):
    printjson(j)

In [34]:
def post(name,url,message,params=None):
    if params is None:
        response = requests.post(url,json=message)
    else:
        response = requests.post(url,json=message,params=params)
    if not response.status_code == 200:
        print(name, 'error:',response.status_code)
        print(response.json())
        return {}
    return response.json()

def automat(db,message):
    automat_url = f'https://automat.renci.org/{db}/query'
    response = requests.post(automat_url,json=message)
    print(response.status_code)
    return response.json()

def strider(message):
    url = 'https://strider.renci.org/query?log_level=DEBUG'
    strider_answer = post(strider,url,message)
    return strider_answer

def aragorn(message, coalesce_type='xnone'):
    if coalesce_type == 'xnone':
        answer = post('aragorn','https://aragorn.renci.org/query',message)
    else:
        answer = post('aragorn','https://aragorn.renci.org/query',message, params={'answer_coalesce_type':coalesce_type})
    return answer

##

def bte(message):
    url = 'https://api.bte.ncats.io/v1/query'
    return post(strider,url,message)

def coalesce(message,method='all'):
    url = 'https://answercoalesce.renci.org/coalesce/graph'
    return post('AC'+method,url,message)

def striderandfriends(message):
    strider_answer = strider(message)    
    coalesced_answer = post('coalesce','https://answercoalesce.renci.org/coalesce/all',strider_answer)
    omni_answer = post('omnicorp','https://aragorn-ranker.renci.org/omnicorp_overlay',coalesced_answer)
    weighted_answer = post('weight','https://aragorn-ranker.renci.org/weight_correctness',omni_answer)
    scored_answer = post('score','https://aragorn-ranker.renci.org/score',weighted_answer)
    return strider_answer,coalesced_answer,omni_answer,weighted_answer,scored_answer

In [35]:
def print_errors(strider_result):
    errorcounts = defaultdict(int)
    for logmessage in strider_result['logs']:
        if logmessage['level'] == 'ERROR':
            jm = json.loads(logmessage['message'])
            words = jm['error'].split()
            e = " ".join(words[:-5])
            errorcounts[e] += 1
    for error,count in errorcounts.items():
        print(f'{error} ({count} times)')
        
def print_queried_sources(strider_result):
    querycounts = defaultdict(int)
    for logmessage in strider_result['logs']:
        j = json.loads(logmessage['message'])
        if 'url' in j:
            querycounts[j['url']] += 1
    for url,count in querycounts.items():
        print(f'{url} ({count} times)')
        
def print_query_for_source(strider_result,url):
    for logmessage in strider_result['logs']:
        j = json.loads(logmessage['message'])
        if 'url' in j and j['url'] == url:
            print(j)
            
def print_results_for_source(strider_result,url):
    for logmessage in strider_result['logs']:
        j = json.loads(logmessage['message'])
        if 'url' in j and j['url'] == url:
            req = j['request']
            resp = j['response']
            print(req['message']['query_graph'])
            print(resp['message']['results'])
            print("")

In [36]:
def retrieve_ars_results(mid):
    message_url = f'https://ars.transltr.io/ars/api/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    results = {}
    for child in j['children']:
        if child['actor']['agent'] in ['ara-aragorn', 'ara-aragorn-exp']:
            childmessage_id = child['message']
            child_url = f'https://ars.transltr.io/ars/api/messages/{childmessage_id}'
            child_response = requests.get(child_url).json()
            try:
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except:
                nresults=0
            print( child['status'], child['actor']['agent'],nresults )
    return results

In [37]:
def get_provenance(message):
    """Given a message with results, find the source of the edges"""
    prov = defaultdict(lambda: defaultdict(int)) # {qedge->{source->count}}
    results = message['message']['results']
    kg = message['message']['knowledge_graph']['edges']
    edge_bindings = [ r['edge_bindings'] for r in results ]
    for bindings in edge_bindings:
        for qg_e, kg_l in bindings.items():
            for kg_e in kg_l:
                for att in kg[kg_e['id']]['attributes']:
                    if att['name'] == 'provenance':
                        source = att['value']
                        prov[qg_e][source]+=1
    qg_edges = []
    sources = []
    counts = []
    for qg_e in prov:
        for source in prov[qg_e]:
            qg_edges.append(qg_e)
            sources.append(source)
            counts.append(prov[qg_e][source])
    prov_table = pd.DataFrame({"QG Edge":qg_edges, "Source":sources, "Count":counts})
    return prov_table

## Query Specific

In [38]:
standup_json='StandupDefinitions/standup_2.json'

In [39]:
with open(standup_json,'r') as jsonfile:
    standup_info = json.load(jsonfile)

In [40]:
display(Markdown(f"# {standup_info['Query Title']}"))
display(Markdown(f"{standup_info['Query Description']}"))
print(f'Github Issue: {standup_info["github_issue"]}')

# 2. Chemicals to Gene

What chemicals are associated with SMS (UniProtKB:P52788)

Github Issue: https://github.com/NCATSTranslator/testing/issues/10


The query as run through the ARS:

In [41]:
query = json.loads(requests.get(standup_info['query_location']).content)
printjson(query)

{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "id": "UniProtKB:P52788",
                    "category": "biolink:Gene"
                },
                "n1": {
                    "category": "biolink:ChemicalSubstance"
                }
            },
            "edges": {
                "e01": {
                    "subject": "n0",
                    "object": "n1"
                }
            }
        }
    }
}


## ARS Assessment

In [12]:
ARS_Responses = [(dt.strptime(x['ARS_result_date'],'%Y-%m-%d'),x['ARS_result_id']) for x in standup_info['ARS_Results']]
ARS_Responses.sort()

In [13]:
for ars_date, ars_id in ARS_Responses:
    display(Markdown(f'### {ars_date}'))
    _ = retrieve_ars_results(ars_id)
    print(f'https://arax.ncats.io/?source=ARS&id={ars_id}')

### 2021-03-02 00:00:00

Done ara-aragorn 63
Done ara-aragorn-exp 0
https://arax.ncats.io/?source=ARS&id=d187edcd-f4a1-42e8-9248-eee0cb161fdb


### 2021-03-16 00:00:00

KeyboardInterrupt: 

## Strider Direct

In [42]:
start = dt.now()
strider_result = strider(query)
end = dt.now()
print(f"Strider produced {len(strider_result['message']['results'])} results in {end-start}.")

Strider produced 37 results in 0:00:27.954131.


### Provenance

In [13]:
prov = get_provenance(strider_result)
display(prov)

,QG Edge,Source,Count
0,e01,https://translator.broadinstitute.org/molepro/...,37


### Queried sources

In [14]:
print_queried_sources(strider_result)

https://automat.renci.org/robokopkg/reasonerapi (84 times)
https://automat.renci.org/cord19-scibite/reasonerapi (2 times)
https://automat.renci.org/kegg/reasonerapi (2 times)
https://automat.renci.org/ctd/reasonerapi (75 times)
https://automat.renci.org/covidkopkg/reasonerapi (84 times)
https://stars-app.renci.org/cam-kp/query (18 times)
https://translator.broadinstitute.org/molepro/trapi/v1.0/query (2 times)
https://automat.renci.org/pharos/reasonerapi (8 times)
https://automat.renci.org/mychem/reasonerapi (16 times)
https://automat.renci.org/chembio/reasonerapi (2 times)
https://automat.renci.org/cord19-scigraph/reasonerapi (4 times)
https://automat.renci.org/hmdb/reasonerapi (2 times)
https://automat.renci.org/gtopdb/reasonerapi (1 times)


### Errors

In [15]:
print_errors(strider_result)

### Results

In [16]:
view = GammaViewer(props={"data":strider_result})
display(view)

### Strider Assessment

Only molpro returns results, though there are many other valid sources.

We are overwhelming BTE.
See: 
https://github.com/RENCI-AUTOMAT/Automat-server/issues/5
https://github.com/ranking-agent/strider/issues/149


Even though there are some correct results in Automats, and strider is getting them, they're not in the results.
See: 
https://github.com/ranking-agent/strider/issues/165

CAM-KP is returning non-interpretable results. (as of 5/1, is throwing a 400).

See:
https://github.com/NCATS-Tangerine/cam-kp-api/issues/257

In [19]:
print_results_for_source(strider_result,'https://automat.renci.org/pharos/reasonerapi')

{'nodes': {'n0': {'category': 'biolink:Gene', 'is_set': False, 'id': 'NCBIGene:6611'}, 'n1': {'category': 'biolink:ChemicalSubstance', 'is_set': False}}, 'edges': {'e01': {'subject': 'n1', 'object': 'n0', 'predicate': 'biolink:condition_associated_with_gene'}}}
[]

{'nodes': {'n0': {'category': 'biolink:Gene', 'is_set': False, 'id': 'NCBIGene:6611'}, 'n1': {'category': 'biolink:ChemicalSubstance', 'is_set': False}}, 'edges': {'e01': {'subject': 'n1', 'object': 'n0', 'predicate': 'biolink:increases_activity_of'}}}
[]

{'nodes': {'n0': {'category': 'biolink:Gene', 'is_set': False, 'id': 'NCBIGene:6611'}, 'n1': {'category': 'biolink:ChemicalSubstance', 'is_set': False}}, 'edges': {'e01': {'subject': 'n0', 'object': 'n1', 'predicate': 'biolink:gene_associated_with_condition'}}}
[]

{'nodes': {'n0': {'category': 'biolink:Gene', 'is_set': False, 'id': 'NCBIGene:6611'}, 'n1': {'category': 'biolink:ChemicalSubstance', 'is_set': False}}, 'edges': {'e01': {'subject': 'n0', 'object': 'n1', 'predi

In [24]:
for n in strider_result['message']['knowledge_graph']['nodes']:
    print (n)

PUBCHEM.COMPOUND:5714
NCBIGene:6611
PUBCHEM.COMPOUND:68911
PUBCHEM.COMPOUND:6708778
PUBCHEM.COMPOUND:122150
PUBCHEM.COMPOUND:44607360
PUBCHEM.COMPOUND:44589607
PUBCHEM.COMPOUND:119828
PUBCHEM.COMPOUND:36462
PUBCHEM.COMPOUND:4178
PUBCHEM.COMPOUND:3926765
PUBCHEM.COMPOUND:439533
PUBCHEM.COMPOUND:932
PUBCHEM.COMPOUND:72390
PUBCHEM.COMPOUND:9865515
PUBCHEM.COMPOUND:5573
PUBCHEM.COMPOUND:65057
PUBCHEM.COMPOUND:25260757
PUBCHEM.COMPOUND:5788723
PUBCHEM.COMPOUND:10219702
PUBCHEM.COMPOUND:4075
PUBCHEM.COMPOUND:65833
PUBCHEM.COMPOUND:16046239
PUBCHEM.COMPOUND:10666
PUBCHEM.COMPOUND:107867
PUBCHEM.COMPOUND:5284550
PUBCHEM.COMPOUND:3335
PUBCHEM.COMPOUND:1103
PUBCHEM.COMPOUND:11957586
PUBCHEM.COMPOUND:11977753
PUBCHEM.COMPOUND:5597
PUBCHEM.COMPOUND:3799
PUBCHEM.COMPOUND:114924
PUBCHEM.COMPOUND:328236
PUBCHEM.COMPOUND:5311281
PUBCHEM.COMPOUND:60838
PUBCHEM.COMPOUND:44632017
PUBCHEM.COMPOUND:5330286


## ARAGORN 

In [29]:
start = dt.now()
aragorn_result = aragorn(query)
end = dt.now()
if 'results' in aragorn_result['message']:
    print(f"ARAGORN produced {len(aragorn_result['message']['results'])} results in {end-start}.")
else:
    print('Error, no result field')

ARAGORN produced 58 results in 0:00:15.812690.


In [30]:
view = GammaViewer(props={"data":aragorn_result})
display(view)

### ARAGORN Assessment

1. No omnicorp results

See: 

https://github.com/TranslatorSRI/NodeNormalization/issues/43

https://github.com/ranking-agent/aragorn-ranker/issues/12

2. No coalescence
This seems to be because of out-of-date databases.

See:

https://github.com/ranking-agent/AnswerCoalesce/issues/44

https://github.com/ranking-agent/strider/issues/164


In [32]:
omni_answer = post('omnicorp','https://aragorn-ranker.renci.org/omnicorp_overlay',strider_result)

omnicorp error: 500


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
omni_answer

In [35]:
coal = post('coalesce','https://answercoalesce.renci.org/coalesce/all',{'message':strider_result['message']})

In [40]:
with open('AC_result_breaks_GV.json.txt','w') as outf:
    json.dump(coal,outf,indent=2)

In [38]:
view = GammaViewer(props={"data":coal})
display(view)

In [101]:
print(coal['message']['results'][38]['node_bindings']['extra_qn_0'])

[{'id': 'MONDO:0000744'}]


In [ ]:
strider_result['message']['knowledge_graph']['nodes']

In [ ]:
q2 = {
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "category": "biolink:Gene",
                    "id": "NCBIGene:6611"
                },
                "n1": {
                    "category": "biolink:ChemicalSubstance"
                }
            },
            "edges": {
                "e01": {
                    "subject": "n0",
                    "object": "n1",
                    "predicate": "biolink:related_to"
                }
            }
        }
    }
}

In [ ]:
r = automat('pharos',q2)

In [ ]:
r['message']['knowledge_graph']['edges']

In [20]:
m ={'query_graph': {'nodes': {'n0': {'id': ['UniProtKB:P52788'], 'category': ['biolink:Gene'], 'is_set': False}, 'n1': {'category': ['biolink:ChemicalSubstance'], 'is_set': True}, 'extra_qn_0': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:MolecularEntity', 'biolink:Gene', 'biolink:GeneOrGeneProduct', 'biolink:MacromolecularMachine', 'biolink:GenomicEntity'], 'is_set': False}}, 'edges': {'e01': {'subject': 'n0', 'object': 'n1'}, 'extra_qe_0': {'subject': 'extra_qn_0', 'object': 'n1'}}}, 'knowledge_graph': {'nodes': {'PUBCHEM.COMPOUND:5714': {'category': ['biolink:ChemicalSubstance'], 'name': '2-(3,4-Dimethoxyphenyl)-n-[2-(3,4-dimethoxyphenyl)ethyl]-n-methylethanamine', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'NCBIGene:6611': {'category': ['biolink:Gene'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 5623, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:68911': {'category': ['biolink:ChemicalSubstance'], 'name': 'Artemether', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:6708778': {'category': ['biolink:ChemicalSubstance'], 'name': '(8Ar,9R)-5-[[(2R,4aR,6R,7R,8R)-7,8-dihydroxy-2-thiophen-2-yl-4,4a,6,7,8,8a-hexahydropyrano[3,2-d][1,3]dioxin-6-yl]oxy]-9-(4-hydroxy-3,5-dimethoxyphenyl)-5a,6,8a,9-tetrahydro-5H-isobenzofuro[6,5-f][1,3]benzodioxol-8-one', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:122150': {'category': ['biolink:ChemicalSubstance'], 'name': 'Saclofen', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:44607360': {'category': ['biolink:ChemicalSubstance'], 'name': 'N-[3-[[5-Chloro-2-[4-(4-methyl-1-piperazinyl)anilino]-4-pyrimidinyl]oxy]phenyl]-2-propenamide', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:44589607': {'category': ['biolink:ChemicalSubstance'], 'name': 'Ethyl (Z)-3-[5-(3-chlorophenyl)tetrazol-2-yl]prop-2-enoate', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:119828': {'category': ['biolink:ChemicalSubstance'], 'name': 'Parecoxib', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:36462': {'category': ['biolink:ChemicalSubstance'], 'name': 'Etoposide', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:4178': {'category': ['biolink:ChemicalSubstance'], 'name': 'Mexiletine', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:3926765': {'category': ['biolink:ChemicalSubstance'], 'name': 'N-[4-(9-Acridinylmethyl)-3-methoxyphenyl]methanesulfonamide', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:439533': {'category': ['biolink:ChemicalSubstance'], 'name': 'Taxifolin', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:932': {'category': ['biolink:ChemicalSubstance'], 'name': '5,7-Dihydroxy-2-(4-hydroxyphenyl)chroman-4-one', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:72390': {'category': ['biolink:ChemicalSubstance'], 'name': 'Procysteine', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:9865515': {'category': ['biolink:ChemicalSubstance'], 'name': 'Mocetinostat', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:5573': {'category': ['biolink:ChemicalSubstance'], 'name': 'Trimebutine', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:65057': {'category': ['biolink:ChemicalSubstance'], 'name': 'Diphenylcyclopropenone', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:25260757': {'category': ['biolink:ChemicalSubstance'], 'name': "N-[4-[1-(1,4-Dioxaspiro[4.5]dec-8-yl)-4-(8-oxa-3-azabicyclo[3.2.1]oct-3-yl)-1H-pyrazolo[3,4-d]pyrimidin-6-yl]phenyl]-N'-methylurea", 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:5788723': {'category': ['biolink:ChemicalSubstance'], 'name': '1-(2-(Diphenylmethoxy)ethyl)-4-(3-phenyl-2-propenyl)piperazine', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:10219702': {'category': ['biolink:ChemicalSubstance'], 'name': 'CID 10219702', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:4075': {'category': ['biolink:ChemicalSubstance'], 'name': 'Mesalamine', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:65833': {'category': ['biolink:ChemicalSubstance'], 'name': 'Milnacipran', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:16046239': {'category': ['biolink:ChemicalSubstance'], 'name': '(3-Bromo-2,5-dimethoxy-7-bicyclo[4.2.0]octa-1(6),2,4-trienyl)methanamine', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:10666': {'category': ['biolink:ChemicalSubstance'], 'name': 'Ricinine', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:107867': {'category': ['biolink:ChemicalSubstance'], 'name': 'Otenzepad', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:5284550': {'category': ['biolink:ChemicalSubstance'], 'name': 'Dosulepin', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:3335': {'category': ['biolink:ChemicalSubstance'], 'name': 'Fenbufen', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:1103': {'category': ['biolink:ChemicalSubstance'], 'name': 'Spermine', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:11957586': {'category': ['biolink:ChemicalSubstance'], 'name': '(2S,3R,5R)-3,4-Dihydroxy-5-[6-[(3-iodophenyl)methylamino]-9-purinyl]-N-methyl-2-oxolanecarboxamide', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:11977753': {'category': ['biolink:ChemicalSubstance'], 'name': 'Dactolisib', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:5597': {'category': ['biolink:ChemicalSubstance'], 'name': 'Troxipide', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:3799': {'category': ['biolink:ChemicalSubstance'], 'name': '1-(2-Naphthalenyl)-2-propen-1-one', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:114924': {'category': ['biolink:ChemicalSubstance'], 'name': 'Tetradecylthioacetic acid', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:328236': {'category': ['biolink:ChemicalSubstance'], 'name': '9-(2,3-Dihydroxy-3-methylbutoxy)furo[3,2-g]chromen-7-one', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:5311281': {'category': ['biolink:ChemicalSubstance'], 'name': 'Cytochalasin B', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:60838': {'category': ['biolink:ChemicalSubstance'], 'name': 'Irinotecan', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:44632017': {'category': ['biolink:ChemicalSubstance'], 'name': '4,7-Dichloro-3-hydroxy-3-[2-(4-methoxyphenyl)-2-oxoethyl]-1H-indol-2-one', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'PUBCHEM.COMPOUND:5330286': {'category': ['biolink:ChemicalSubstance'], 'name': 'Palbociclib', 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'NCBIGENE:1576': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:MolecularEntity', 'biolink:Gene', 'biolink:GeneOrGeneProduct', 'biolink:MacromolecularMachine', 'biolink:GenomicEntity'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'MONDO:0000744': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:Disease', 'biolink:DiseaseOrPhenotypicFeature'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 5983, 'name': 'omnicorp_article_count'}]}, 'MONDO:0000709': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:Disease', 'biolink:DiseaseOrPhenotypicFeature'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 2762, 'name': 'omnicorp_article_count'}]}, 'MONDO:0005365': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:Disease', 'biolink:DiseaseOrPhenotypicFeature'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 76485, 'name': 'omnicorp_article_count'}]}, 'MONDO:0019121': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:Disease', 'biolink:DiseaseOrPhenotypicFeature'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 22097, 'name': 'omnicorp_article_count'}]}, 'MONDO:0001673': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:Disease', 'biolink:DiseaseOrPhenotypicFeature'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 120408, 'name': 'omnicorp_article_count'}]}, 'MONDO:0001442': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:Disease', 'biolink:DiseaseOrPhenotypicFeature'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 3197, 'name': 'omnicorp_article_count'}]}, 'NCBIGENE:9429': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:MolecularEntity', 'biolink:Gene', 'biolink:GeneOrGeneProduct', 'biolink:MacromolecularMachine', 'biolink:GenomicEntity'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'MONDO:0005335': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:Disease', 'biolink:DiseaseOrPhenotypicFeature'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 85879, 'name': 'omnicorp_article_count'}]}, 'NCBIGENE:1555': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:MolecularEntity', 'biolink:Gene', 'biolink:GeneOrGeneProduct', 'biolink:MacromolecularMachine', 'biolink:GenomicEntity'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'NCBIGENE:5925': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:MolecularEntity', 'biolink:Gene', 'biolink:GeneOrGeneProduct', 'biolink:MacromolecularMachine', 'biolink:GenomicEntity'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'MONDO:0018874': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:Disease', 'biolink:DiseaseOrPhenotypicFeature'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 60837, 'name': 'omnicorp_article_count'}]}, 'MONDO:0016812': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:Disease', 'biolink:DiseaseOrPhenotypicFeature'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 13307, 'name': 'omnicorp_article_count'}]}, 'NCBIGENE:54658': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:MolecularEntity', 'biolink:Gene', 'biolink:GeneOrGeneProduct', 'biolink:MacromolecularMachine', 'biolink:GenomicEntity'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'NCBIGENE:7153': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:MolecularEntity', 'biolink:Gene', 'biolink:GeneOrGeneProduct', 'biolink:MacromolecularMachine', 'biolink:GenomicEntity'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'UMLS:C0854142': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:DiseaseOrPhenotypicFeature', 'biolink:PhenotypicFeature'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'HP:0002013': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:DiseaseOrPhenotypicFeature', 'biolink:PhenotypicFeature'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 60288, 'name': 'omnicorp_article_count'}]}, 'MONDO:0005246': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:Disease', 'biolink:DiseaseOrPhenotypicFeature'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 28264, 'name': 'omnicorp_article_count'}]}, 'NCBIGENE:355': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:MolecularEntity', 'biolink:Gene', 'biolink:GeneOrGeneProduct', 'biolink:MacromolecularMachine', 'biolink:GenomicEntity'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'UMLS:C0854327': {'category': ['biolink:NamedThing', 'biolink:BiologicalEntity', 'biolink:Disease', 'biolink:DiseaseOrPhenotypicFeature'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}, 'UMLS:C0854471': {'category': ['biolink:NamedThing'], 'attributes': [{'type': 'EDAM:data_0006', 'value': 0, 'name': 'omnicorp_article_count'}]}}, 'edges': {'0d2ef096f111': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:5597', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '8c92bddf620d': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:5330286', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, 'e6bc3a4d1c53': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:60838', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '5c12b356a2cd': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:5311281', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, 'b4c033149583': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:328236', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '264dfd28824a': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:65833', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, 'e1b17345df8b': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:36462', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, 'b7f6e6bedb80': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:439533', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '62b2b4d39c01': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:44589607', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '6dd4086c9e7a': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:4178', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '29e0bff11ca3': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:932', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '8a23065bea55': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:68911', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '51f52f21e5ee': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:3799', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, 'a9af89d1b48e': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:5714', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '6e44ebaae2d5': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:44632017', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '5a672d9f196e': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:44607360', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '653a38a2676a': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:5788723', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '6740f8b5fad2': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:10666', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, 'e1e214f02c09': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:3926765', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '544c025e519f': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:65057', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '8ee217e1b930': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:119828', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, 'b24926d3d132': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:72390', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '2661c8b12c3c': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:122150', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '9ef01671dfec': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:4075', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, 'f00b8f979a52': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:107867', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, 'e29597429265': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:11977753', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '2a6ad9515281': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:9865515', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, 'cb427f094e10': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:10219702', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '8f92dbb28ca1': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:5573', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, 'a1ae0b1aa7b0': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:114924', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '09d94868f616': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:3335', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '9466f00f0740': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:25260757', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, 'd67d490a5eab': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:5284550', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '07c5bf8af455': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:11957586', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '77a91b7f5815': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:1103', 'predicate': 'biolink:affected_by', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '7781173d73cc': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:16046239', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '5f3ce2525a03': {'subject': 'NCBIGene:6611', 'object': 'PUBCHEM.COMPOUND:6708778', 'predicate': 'biolink:correlated_with', 'attributes': [{'type': 'MetaInformation', 'value': 'https://translator.broadinstitute.org/molepro/trapi/v1.0/query', 'name': 'provenance'}]}, '-7687093198747123667': {'subject': 'NCBIGENE:1576', 'object': 'PUBCHEM.COMPOUND:119828', 'predicate': 'biolink:increases_degradation_of'}, '-1940792003355207243': {'subject': 'NCBIGENE:1576', 'object': 'PUBCHEM.COMPOUND:60838', 'predicate': 'biolink:increases_degradation_of'}, '8233649074718425276': {'subject': 'NCBIGENE:1576', 'object': 'PUBCHEM.COMPOUND:68911', 'predicate': 'biolink:increases_degradation_of'}, '-3322974684512804495': {'subject': 'NCBIGENE:1576', 'object': 'PUBCHEM.COMPOUND:36462', 'predicate': 'biolink:increases_degradation_of'}, '-3071087844585659969': {'subject': 'NCBIGENE:1576', 'object': 'PUBCHEM.COMPOUND:5573', 'predicate': 'biolink:increases_degradation_of'}, '8317526272471970482': {'subject': 'NCBIGENE:1576', 'object': 'PUBCHEM.COMPOUND:4178', 'predicate': 'biolink:increases_degradation_of'}, '-9145375113075338147': {'subject': 'NCBIGENE:1576', 'object': 'PUBCHEM.COMPOUND:5330286', 'predicate': 'biolink:increases_degradation_of'}, '-2653782951985685620': {'subject': 'PUBCHEM.COMPOUND:36462', 'object': 'MONDO:0000744', 'predicate': 'biolink:causes_adverse_event'}, '6415052928970676496': {'subject': 'PUBCHEM.COMPOUND:4075', 'object': 'MONDO:0000744', 'predicate': 'biolink:causes_adverse_event'}, '3148454403947782132': {'subject': 'PUBCHEM.COMPOUND:60838', 'object': 'MONDO:0000744', 'predicate': 'biolink:causes_adverse_event'}, '8535480063114958772': {'subject': 'PUBCHEM.COMPOUND:36462', 'object': 'MONDO:0000709', 'predicate': 'biolink:causes_adverse_event'}, '6296572583226592264': {'subject': 'PUBCHEM.COMPOUND:4075', 'object': 'MONDO:0000709', 'predicate': 'biolink:causes_adverse_event'}, '2893698903015109452': {'subject': 'PUBCHEM.COMPOUND:60838', 'object': 'MONDO:0000709', 'predicate': 'biolink:causes_adverse_event'}, '-7655930571997630742': {'subject': 'PUBCHEM.COMPOUND:36462', 'object': 'MONDO:0005365', 'predicate': 'biolink:prevents'}, '-2945808497405411738': {'subject': 'PUBCHEM.COMPOUND:4075', 'object': 'MONDO:0005365', 'predicate': 'biolink:prevents'}, '2398998877798963234': {'subject': 'PUBCHEM.COMPOUND:60838', 'object': 'MONDO:0005365', 'predicate': 'biolink:prevents'}, '-5255873100157230963': {'subject': 'PUBCHEM.COMPOUND:36462', 'object': 'MONDO:0019121', 'predicate': 'biolink:causes_adverse_event'}, '-2554269671851742903': {'subject': 'PUBCHEM.COMPOUND:4075', 'object': 'MONDO:0019121', 'predicate': 'biolink:causes_adverse_event'}, '7307485404034861093': {'subject': 'PUBCHEM.COMPOUND:60838', 'object': 'MONDO:0019121', 'predicate': 'biolink:causes_adverse_event'}, '-4782887461530644258': {'subject': 'PUBCHEM.COMPOUND:36462', 'object': 'MONDO:0001673', 'predicate': 'biolink:contributes_to'}, '-5181821415067915534': {'subject': 'PUBCHEM.COMPOUND:4075', 'object': 'MONDO:0001673', 'predicate': 'biolink:contributes_to'}, '-5274006731422776743': {'subject': 'PUBCHEM.COMPOUND:5330286', 'object': 'MONDO:0001673', 'predicate': 'biolink:contributes_to'}, '-6643214779650985450': {'subject': 'PUBCHEM.COMPOUND:60838', 'object': 'MONDO:0001673', 'predicate': 'biolink:contributes_to'}, '4571139681287114491': {'subject': 'PUBCHEM.COMPOUND:36462', 'object': 'MONDO:0001442', 'predicate': 'biolink:contributes_to'}, '-1132400003378559065': {'subject': 'PUBCHEM.COMPOUND:4075', 'object': 'MONDO:0001442', 'predicate': 'biolink:contributes_to'}, '6672656488345524288': {'subject': 'PUBCHEM.COMPOUND:5330286', 'object': 'MONDO:0001442', 'predicate': 'biolink:contributes_to'}, '6425767663672567235': {'subject': 'PUBCHEM.COMPOUND:60838', 'object': 'MONDO:0001442', 'predicate': 'biolink:contributes_to'}, '-8620808213747174862': {'subject': 'NCBIGENE:9429', 'object': 'PUBCHEM.COMPOUND:36462', 'predicate': 'biolink:increases_degradation_of'}, '7474903407177340294': {'subject': 'NCBIGENE:9429', 'object': 'PUBCHEM.COMPOUND:5330286', 'predicate': 'biolink:increases_degradation_of'}, '-1460443560609723762': {'subject': 'NCBIGENE:9429', 'object': 'PUBCHEM.COMPOUND:60838', 'predicate': 'biolink:increases_degradation_of'}, '4753381425181620203': {'subject': 'PUBCHEM.COMPOUND:36462', 'object': 'MONDO:0005335', 'predicate': 'biolink:treats'}, '-8753709916190274921': {'subject': 'PUBCHEM.COMPOUND:4075', 'object': 'MONDO:0005335', 'predicate': 'biolink:treats'}, '-3574536809280340909': {'subject': 'PUBCHEM.COMPOUND:60838', 'object': 'MONDO:0005335', 'predicate': 'biolink:treats'}, '1786981203419502805': {'subject': 'NCBIGENE:1555', 'object': 'PUBCHEM.COMPOUND:4178', 'predicate': 'biolink:increases_degradation_of'}, '-226216645501054465': {'subject': 'NCBIGENE:1555', 'object': 'PUBCHEM.COMPOUND:68911', 'predicate': 'biolink:increases_degradation_of'}, '6761486950769164754': {'subject': 'NCBIGENE:1555', 'object': 'PUBCHEM.COMPOUND:60838', 'predicate': 'biolink:increases_degradation_of'}, '-1985859302229132407': {'subject': 'NCBIGENE:5925', 'object': 'PUBCHEM.COMPOUND:36462', 'predicate': 'biolink:affects_response_to'}, '-6459804033034556875': {'subject': 'NCBIGENE:5925', 'object': 'PUBCHEM.COMPOUND:5330286', 'predicate': 'biolink:affects_response_to'}, '3821263606095217354': {'subject': 'PUBCHEM.COMPOUND:36462', 'object': 'MONDO:0018874', 'predicate': 'biolink:treats'}, '2595197652355029893': {'subject': 'PUBCHEM.COMPOUND:5330286', 'object': 'MONDO:0018874', 'predicate': 'biolink:treats'}, '8100084659266336770': {'subject': 'PUBCHEM.COMPOUND:60838', 'object': 'MONDO:0018874', 'predicate': 'biolink:treats'}, '2594006064662483700': {'subject': 'PUBCHEM.COMPOUND:36462', 'object': 'MONDO:0016812', 'predicate': 'biolink:prevents'}, '6181877842237597512': {'subject': 'PUBCHEM.COMPOUND:4075', 'object': 'MONDO:0016812', 'predicate': 'biolink:prevents'}, '-8839012501802971508': {'subject': 'PUBCHEM.COMPOUND:60838', 'object': 'MONDO:0016812', 'predicate': 'biolink:prevents'}, '5960923107060258946': {'subject': 'NCBIGENE:54658', 'object': 'PUBCHEM.COMPOUND:36462', 'predicate': 'biolink:affects_response_to'}, '7030709820075254846': {'subject': 'NCBIGENE:54658', 'object': 'PUBCHEM.COMPOUND:60838', 'predicate': 'biolink:affects_response_to'}, '-783881862789761662': {'subject': 'NCBIGENE:7153', 'object': 'PUBCHEM.COMPOUND:36462', 'predicate': 'biolink:increases_response_to'}, '1453315862198051134': {'subject': 'NCBIGENE:7153', 'object': 'PUBCHEM.COMPOUND:60838', 'predicate': 'biolink:increases_response_to'}, '-1479263711107237371': {'subject': 'PUBCHEM.COMPOUND:36462', 'object': 'UMLS:C0854142', 'predicate': 'biolink:causes_adverse_event'}, '7343875279859891821': {'subject': 'PUBCHEM.COMPOUND:60838', 'object': 'UMLS:C0854142', 'predicate': 'biolink:causes_adverse_event'}, '-1188366725817507183': {'subject': 'PUBCHEM.COMPOUND:36462', 'object': 'HP:0002013', 'predicate': 'biolink:contributes_to'}, '-469733590235674203': {'subject': 'PUBCHEM.COMPOUND:4075', 'object': 'HP:0002013', 'predicate': 'biolink:contributes_to'}, '4072795741069439142': {'subject': 'PUBCHEM.COMPOUND:5330286', 'object': 'HP:0002013', 'predicate': 'biolink:contributes_to'}, '-1170087239384795191': {'subject': 'PUBCHEM.COMPOUND:60838', 'object': 'HP:0002013', 'predicate': 'biolink:contributes_to'}, '-4102541268345724316': {'subject': 'PUBCHEM.COMPOUND:36462', 'object': 'MONDO:0005246', 'predicate': 'biolink:prevents'}, '8601128738260250552': {'subject': 'PUBCHEM.COMPOUND:4075', 'object': 'MONDO:0005246', 'predicate': 'biolink:prevents'}, '-87318139188113508': {'subject': 'PUBCHEM.COMPOUND:60838', 'object': 'MONDO:0005246', 'predicate': 'biolink:prevents'}, '6484121316167818226': {'subject': 'PUBCHEM.COMPOUND:36462', 'object': 'NCBIGENE:355', 'predicate': 'biolink:increases_localization_of'}, '-6751169000268224422': {'subject': 'PUBCHEM.COMPOUND:60838', 'object': 'NCBIGENE:355', 'predicate': 'biolink:increases_localization_of'}, '-501190328348089864': {'subject': 'PUBCHEM.COMPOUND:36462', 'object': 'UMLS:C0854327', 'predicate': 'biolink:causes_adverse_event'}, '1036167886288842336': {'subject': 'PUBCHEM.COMPOUND:60838', 'object': 'UMLS:C0854327', 'predicate': 'biolink:causes_adverse_event'}, '3619557085998785399': {'subject': 'PUBCHEM.COMPOUND:36462', 'object': 'UMLS:C0854471', 'predicate': 'biolink:causes_adverse_event'}, '-3488360998789182289': {'subject': 'PUBCHEM.COMPOUND:60838', 'object': 'UMLS:C0854471', 'predicate': 'biolink:causes_adverse_event'}, 'ad14689e-4b0a-479b-936a-a3326655533f': {'subject': 'MONDO:0000744', 'object': 'NCBIGene:6611', 'predicate': 'biolink:literature_co_occurrence', 'attributes': [{'type': 'num_publications', 'value': 5}, {'type': 'publications', 'value': []}, {'type': 'source_database', 'value': 'omnicorp'}, {'type': 'edge_source', 'value': 'omnicorp.term_to_term'}]}, 'bdb51126-0677-45b2-8136-044aac61454e': {'subject': 'MONDO:0005365', 'object': 'NCBIGene:6611', 'predicate': 'biolink:literature_co_occurrence', 'attributes': [{'type': 'num_publications', 'value': 52}, {'type': 'publications', 'value': []}, {'type': 'source_database', 'value': 'omnicorp'}, {'type': 'edge_source', 'value': 'omnicorp.term_to_term'}]}, '1dbb49d0-0220-44b0-a9b5-7873af96d4dc': {'subject': 'MONDO:0019121', 'object': 'NCBIGene:6611', 'predicate': 'biolink:literature_co_occurrence', 'attributes': [{'type': 'num_publications', 'value': 28}, {'type': 'publications', 'value': []}, {'type': 'source_database', 'value': 'omnicorp'}, {'type': 'edge_source', 'value': 'omnicorp.term_to_term'}]}, '1418a8a7-f59b-4284-8d1f-ed827d2fe03d': {'subject': 'MONDO:0001673', 'object': 'NCBIGene:6611', 'predicate': 'biolink:literature_co_occurrence', 'attributes': [{'type': 'num_publications', 'value': 172}, {'type': 'publications', 'value': []}, {'type': 'source_database', 'value': 'omnicorp'}, {'type': 'edge_source', 'value': 'omnicorp.term_to_term'}]}, '9edd07f5-4b9f-4908-becd-827985cc77f2': {'subject': 'MONDO:0005335', 'object': 'NCBIGene:6611', 'predicate': 'biolink:literature_co_occurrence', 'attributes': [{'type': 'num_publications', 'value': 13}, {'type': 'publications', 'value': []}, {'type': 'source_database', 'value': 'omnicorp'}, {'type': 'edge_source', 'value': 'omnicorp.term_to_term'}]}, '1c036760-5fee-460f-86d1-6402b20a3f5c': {'subject': 'MONDO:0018874', 'object': 'NCBIGene:6611', 'predicate': 'biolink:literature_co_occurrence', 'attributes': [{'type': 'num_publications', 'value': 34}, {'type': 'publications', 'value': []}, {'type': 'source_database', 'value': 'omnicorp'}, {'type': 'edge_source', 'value': 'omnicorp.term_to_term'}]}, '77fc25a7-b458-4f68-adab-97ac48a7ee3f': {'subject': 'MONDO:0016812', 'object': 'NCBIGene:6611', 'predicate': 'biolink:literature_co_occurrence', 'attributes': [{'type': 'num_publications', 'value': 8}, {'type': 'publications', 'value': []}, {'type': 'source_database', 'value': 'omnicorp'}, {'type': 'edge_source', 'value': 'omnicorp.term_to_term'}]}, '0685ebf0-556b-48fe-9ca7-8e3ae584f774': {'subject': 'HP:0002013', 'object': 'NCBIGene:6611', 'predicate': 'biolink:literature_co_occurrence', 'attributes': [{'type': 'num_publications', 'value': 75}, {'type': 'publications', 'value': []}, {'type': 'source_database', 'value': 'omnicorp'}, {'type': 'edge_source', 'value': 'omnicorp.term_to_term'}]}, '249b73db-8653-44d3-b9cc-524555a0d9ca': {'subject': 'MONDO:0005246', 'object': 'NCBIGene:6611', 'predicate': 'biolink:literature_co_occurrence', 'attributes': [{'type': 'num_publications', 'value': 20}, {'type': 'publications', 'value': []}, {'type': 'source_database', 'value': 'omnicorp'}, {'type': 'edge_source', 'value': 'omnicorp.term_to_term'}]}}}, 'results': [{'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:119828', 'p_value': 3.904388366202427e-11, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:1576']}, {'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 3.904388366202427e-11, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:1576']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 3.904388366202427e-11, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:1576']}, {'id': 'PUBCHEM.COMPOUND:5573', 'p_value': 3.904388366202427e-11, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:1576']}, {'id': 'PUBCHEM.COMPOUND:4178', 'p_value': 3.904388366202427e-11, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:1576']}, {'id': 'PUBCHEM.COMPOUND:5330286', 'p_value': 3.904388366202427e-11, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:1576']}, {'id': 'PUBCHEM.COMPOUND:68911', 'p_value': 3.904388366202427e-11, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:1576']}], 'extra_qn_0': [{'id': 'NCBIGENE:1576'}]}, 'edge_bindings': {'e01': [{'id': '8ee217e1b930'}, {'id': '6dd4086c9e7a'}, {'id': '8c92bddf620d'}, {'id': '8a23065bea55'}, {'id': 'e1b17345df8b'}, {'id': '8f92dbb28ca1'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '-1940792003355207243'}, {'id': '-3322974684512804495'}, {'id': '-9145375113075338147'}, {'id': '-3071087844585659969'}, {'id': '8317526272471970482'}, {'id': '8233649074718425276'}, {'id': '-7687093198747123667'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 2.8395452584691076e-08, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0000744']}, {'id': 'PUBCHEM.COMPOUND:4075', 'p_value': 2.8395452584691076e-08, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0000744']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 2.8395452584691076e-08, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0000744']}], 'extra_qn_0': [{'id': 'MONDO:0000744'}]}, 'edge_bindings': {'e01': [{'id': '9ef01671dfec'}, {'id': 'e1b17345df8b'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '3148454403947782132'}, {'id': '-2653782951985685620'}, {'id': '6415052928970676496'}], 's15': [{'id': 'ad14689e-4b0a-479b-936a-a3326655533f'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 6.2742898231296e-08, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0000709']}, {'id': 'PUBCHEM.COMPOUND:4075', 'p_value': 6.2742898231296e-08, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0000709']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 6.2742898231296e-08, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0000709']}], 'extra_qn_0': [{'id': 'MONDO:0000709'}]}, 'edge_bindings': {'e01': [{'id': '9ef01671dfec'}, {'id': 'e1b17345df8b'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '2893698903015109452'}, {'id': '8535480063114958772'}, {'id': '6296572583226592264'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 1.240903469462318e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0005365']}, {'id': 'PUBCHEM.COMPOUND:4075', 'p_value': 1.240903469462318e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0005365']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 1.240903469462318e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0005365']}], 'extra_qn_0': [{'id': 'MONDO:0005365'}]}, 'edge_bindings': {'e01': [{'id': '9ef01671dfec'}, {'id': 'e1b17345df8b'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '-2945808497405411738'}, {'id': '-7655930571997630742'}, {'id': '2398998877798963234'}], 's24': [{'id': 'bdb51126-0677-45b2-8136-044aac61454e'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 1.3109617450518593e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0019121']}, {'id': 'PUBCHEM.COMPOUND:4075', 'p_value': 1.3109617450518593e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0019121']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 1.3109617450518593e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0019121']}], 'extra_qn_0': [{'id': 'MONDO:0019121'}]}, 'edge_bindings': {'e01': [{'id': '9ef01671dfec'}, {'id': 'e1b17345df8b'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '-2554269671851742903'}, {'id': '7307485404034861093'}, {'id': '-5255873100157230963'}], 's28': [{'id': '1dbb49d0-0220-44b0-a9b5-7873af96d4dc'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 2.0033567084199524e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0001673']}, {'id': 'PUBCHEM.COMPOUND:4075', 'p_value': 2.0033567084199524e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0001673']}, {'id': 'PUBCHEM.COMPOUND:5330286', 'p_value': 2.0033567084199524e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0001673']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 2.0033567084199524e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0001673']}], 'extra_qn_0': [{'id': 'MONDO:0001673'}]}, 'edge_bindings': {'e01': [{'id': '9ef01671dfec'}, {'id': '8c92bddf620d'}, {'id': 'e1b17345df8b'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '-4782887461530644258'}, {'id': '-5181821415067915534'}, {'id': '-6643214779650985450'}, {'id': '-5274006731422776743'}], 's32': [{'id': '1418a8a7-f59b-4284-8d1f-ed827d2fe03d'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 2.596027839109558e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0001442']}, {'id': 'PUBCHEM.COMPOUND:4075', 'p_value': 2.596027839109558e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0001442']}, {'id': 'PUBCHEM.COMPOUND:5330286', 'p_value': 2.596027839109558e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0001442']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 2.596027839109558e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0001442']}], 'extra_qn_0': [{'id': 'MONDO:0001442'}]}, 'edge_bindings': {'e01': [{'id': '9ef01671dfec'}, {'id': '8c92bddf620d'}, {'id': 'e1b17345df8b'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '-1132400003378559065'}, {'id': '6672656488345524288'}, {'id': '4571139681287114491'}, {'id': '6425767663672567235'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 3.87319092905419e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:9429']}, {'id': 'PUBCHEM.COMPOUND:5330286', 'p_value': 3.87319092905419e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:9429']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 3.87319092905419e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:9429']}], 'extra_qn_0': [{'id': 'NCBIGENE:9429'}]}, 'edge_bindings': {'e01': [{'id': '8c92bddf620d'}, {'id': 'e1b17345df8b'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '-1460443560609723762'}, {'id': '7474903407177340294'}, {'id': '-8620808213747174862'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 4.0216367022425543e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0005335']}, {'id': 'PUBCHEM.COMPOUND:4075', 'p_value': 4.0216367022425543e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0005335']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 4.0216367022425543e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0005335']}], 'extra_qn_0': [{'id': 'MONDO:0005335'}]}, 'edge_bindings': {'e01': [{'id': '9ef01671dfec'}, {'id': 'e1b17345df8b'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '-3574536809280340909'}, {'id': '-8753709916190274921'}, {'id': '4753381425181620203'}], 's46': [{'id': '9edd07f5-4b9f-4908-becd-827985cc77f2'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:4178', 'p_value': 4.4895507186915324e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:1555']}, {'id': 'PUBCHEM.COMPOUND:68911', 'p_value': 4.4895507186915324e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:1555']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 4.4895507186915324e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:1555']}], 'extra_qn_0': [{'id': 'NCBIGENE:1555'}]}, 'edge_bindings': {'e01': [{'id': '6dd4086c9e7a'}, {'id': '8a23065bea55'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '1786981203419502805'}, {'id': '-226216645501054465'}, {'id': '6761486950769164754'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 5.090468292891404e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:5925']}, {'id': 'PUBCHEM.COMPOUND:5330286', 'p_value': 5.090468292891404e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:5925']}], 'extra_qn_0': [{'id': 'NCBIGENE:5925'}]}, 'edge_bindings': {'e01': [{'id': '8c92bddf620d'}, {'id': 'e1b17345df8b'}], 'extra_qe_0': [{'id': '-6459804033034556875'}, {'id': '-1985859302229132407'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 6.307683812206184e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0018874']}, {'id': 'PUBCHEM.COMPOUND:5330286', 'p_value': 6.307683812206184e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0018874']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 6.307683812206184e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0018874']}], 'extra_qn_0': [{'id': 'MONDO:0018874'}]}, 'edge_bindings': {'e01': [{'id': '8c92bddf620d'}, {'id': 'e1b17345df8b'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '2595197652355029893'}, {'id': '3821263606095217354'}, {'id': '8100084659266336770'}], 's57': [{'id': '1c036760-5fee-460f-86d1-6402b20a3f5c'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 6.512532135084513e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0016812']}, {'id': 'PUBCHEM.COMPOUND:4075', 'p_value': 6.512532135084513e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0016812']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 6.512532135084513e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0016812']}], 'extra_qn_0': [{'id': 'MONDO:0016812'}]}, 'edge_bindings': {'e01': [{'id': '9ef01671dfec'}, {'id': 'e1b17345df8b'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '2594006064662483700'}, {'id': '-8839012501802971508'}, {'id': '6181877842237597512'}], 's61': [{'id': '77fc25a7-b458-4f68-adab-97ac48a7ee3f'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 6.9279160189106e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:54658']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 6.9279160189106e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:54658']}], 'extra_qn_0': [{'id': 'NCBIGENE:54658'}]}, 'edge_bindings': {'e01': [{'id': 'e1b17345df8b'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '7030709820075254846'}, {'id': '5960923107060258946'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 6.9279160189106e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:7153']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 6.9279160189106e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:7153']}], 'extra_qn_0': [{'id': 'NCBIGENE:7153'}]}, 'edge_bindings': {'e01': [{'id': 'e1b17345df8b'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '-783881862789761662'}, {'id': '1453315862198051134'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 6.9279160189106e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['UMLS:C0854142']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 6.9279160189106e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['UMLS:C0854142']}], 'extra_qn_0': [{'id': 'UMLS:C0854142'}]}, 'edge_bindings': {'e01': [{'id': 'e1b17345df8b'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '7343875279859891821'}, {'id': '-1479263711107237371'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 7.033262225845803e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['HP:0002013']}, {'id': 'PUBCHEM.COMPOUND:4075', 'p_value': 7.033262225845803e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['HP:0002013']}, {'id': 'PUBCHEM.COMPOUND:5330286', 'p_value': 7.033262225845803e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['HP:0002013']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 7.033262225845803e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['HP:0002013']}], 'extra_qn_0': [{'id': 'HP:0002013'}]}, 'edge_bindings': {'e01': [{'id': '9ef01671dfec'}, {'id': '8c92bddf620d'}, {'id': 'e1b17345df8b'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '-1170087239384795191'}, {'id': '-469733590235674203'}, {'id': '-1188366725817507183'}, {'id': '4072795741069439142'}], 's74': [{'id': '0685ebf0-556b-48fe-9ca7-8e3ae584f774'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 8.312446100680554e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0005246']}, {'id': 'PUBCHEM.COMPOUND:4075', 'p_value': 8.312446100680554e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0005246']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 8.312446100680554e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['MONDO:0005246']}], 'extra_qn_0': [{'id': 'MONDO:0005246'}]}, 'edge_bindings': {'e01': [{'id': '9ef01671dfec'}, {'id': 'e1b17345df8b'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '8601128738260250552'}, {'id': '-87318139188113508'}, {'id': '-4102541268345724316'}], 's79': [{'id': '249b73db-8653-44d3-b9cc-524555a0d9ca'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 9.047691992320636e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:355']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 9.047691992320636e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['NCBIGENE:355']}], 'extra_qn_0': [{'id': 'NCBIGENE:355'}]}, 'edge_bindings': {'e01': [{'id': 'e1b17345df8b'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '6484121316167818226'}, {'id': '-6751169000268224422'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 9.047691992320636e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['UMLS:C0854327']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 9.047691992320636e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['UMLS:C0854327']}], 'extra_qn_0': [{'id': 'UMLS:C0854327'}]}, 'edge_bindings': {'e01': [{'id': 'e1b17345df8b'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '1036167886288842336'}, {'id': '-501190328348089864'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611'}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'p_value': 9.047691992320636e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['UMLS:C0854471']}, {'id': 'PUBCHEM.COMPOUND:60838', 'p_value': 9.047691992320636e-07, 'coalescence_method': 'graph_enrichment', 'enriched_nodes': ['UMLS:C0854471']}], 'extra_qn_0': [{'id': 'UMLS:C0854471'}]}, 'edge_bindings': {'e01': [{'id': 'e1b17345df8b'}, {'id': 'e6bc3a4d1c53'}], 'extra_qe_0': [{'id': '3619557085998785399'}, {'id': '-3488360998789182289'}]}, 'score': 0.0}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:1103', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '77a91b7f5815'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:4178', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '6dd4086c9e7a'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:72390', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': 'b24926d3d132'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:932', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '29e0bff11ca3'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:11977753', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': 'e29597429265'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:10666', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '6740f8b5fad2'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:5284550', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': 'd67d490a5eab'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:16046239', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '7781173d73cc'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:439533', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': 'b7f6e6bedb80'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:60838', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': 'e6bc3a4d1c53'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:11957586', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '07c5bf8af455'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:9865515', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '2a6ad9515281'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:122150', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '2661c8b12c3c'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:44632017', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '6e44ebaae2d5'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:119828', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '8ee217e1b930'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:5597', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '0d2ef096f111'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:44607360', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '5a672d9f196e'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:114924', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': 'a1ae0b1aa7b0'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:6708778', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '5f3ce2525a03'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:3335', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '09d94868f616'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:25260757', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '9466f00f0740'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:3926765', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': 'e1e214f02c09'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:44589607', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '62b2b4d39c01'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:5311281', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '5c12b356a2cd'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:10219702', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': 'cb427f094e10'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:5714', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': 'a9af89d1b48e'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:5788723', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '653a38a2676a'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:65057', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '544c025e519f'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:328236', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': 'b4c033149583'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:36462', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': 'e1b17345df8b'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:5330286', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '8c92bddf620d'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:107867', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': 'f00b8f979a52'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:4075', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '9ef01671dfec'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:65833', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '264dfd28824a'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:68911', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '8a23065bea55'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:3799', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '51f52f21e5ee'}]}}, {'node_bindings': {'n0': [{'id': 'NCBIGene:6611', 'category': ['biolink:Gene']}], 'n1': [{'id': 'PUBCHEM.COMPOUND:5573', 'category': ['biolink:ChemicalSubstance']}]}, 'edge_bindings': {'e01': [{'id': '8f92dbb28ca1'}]}}]}


In [21]:
print_json(m)

{
    "query_graph": {
        "nodes": {
            "n0": {
                "id": [
                    "UniProtKB:P52788"
                ],
                "category": [
                    "biolink:Gene"
                ],
                "is_set": false
            },
            "n1": {
                "category": [
                    "biolink:ChemicalSubstance"
                ],
                "is_set": true
            },
            "extra_qn_0": {
                "category": [
                    "biolink:NamedThing",
                    "biolink:BiologicalEntity",
                    "biolink:MolecularEntity",
                    "biolink:Gene",
                    "biolink:GeneOrGeneProduct",
                    "biolink:MacromolecularMachine",
                    "biolink:GenomicEntity"
                ],
                "is_set": false
            }
        },
        "edges": {
            "e01": {
                "subject": "n0",
                "object": "n1"
     

In [22]:
with open('breaksrank.json','w') as outf:
    json.dump(m,outf)

In [23]:
rq = {'message': m}

In [24]:
weighted_answer = post('weight','https://aragorn-ranker.renci.org/weight_correctness',rq)

In [26]:
len(weighted_answer['message']['results'])

58

In [27]:
scoreded_answer = post('score','https://aragorn-ranker.renci.org/score',weighted_answer)

score error: 500


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [28]:
with open('weighted_ansswer.json.txt','w') as outf:
    json.dump(weighted_answer,outf)